In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from Lib.module.risk_budgeting import *
import student_mixture as sm
from tqdm import tqdm
from Lib.module.movidas import *

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
np.random.seed(0)

In [4]:
freq = 'B'
df_all = pd.concat([pd.read_excel('Data/SP_RC.xlsx',index_col=0, header=[0,1]), pd.read_excel('Data/SP_RC2.xlsx',index_col=0, header=[0,1])])
df_all = df_all.dropna(how='all').dropna(axis=1)
df_all = df_all.pct_change().dropna()
df_all = df_all.replace([np.inf, -np.inf], np.nan)
df_all = df_all[~df_all.index.duplicated(keep='first')]
first_level_index = df_all.columns.get_level_values(0)


In [ ]:
gamma_sgd = {10:2.5, 25:.8, 50:.25, 100:.08, 250:.04}
gamma_tamed = {10:1, 25:2.5, 50:5, 100:10, 250:25}

errors_sgd = {10: [], 25: [], 50: [], 100: [], 250: []}
errors_sgd_tamed = {10: [], 25: [], 50: [], 100: [], 250: []}
errors_smd = {10: [], 25: [], 50: [], 100: [], 250: []}

norms_sgd = {10: [], 25: [], 50: [], 100: [], 250: []}
norms_sgd_tamed = {10: [], 25: [], 50: [], 100: [], 250: []}
norms_smd = {10: [], 25: [], 50: [], 100: [], 250: []}

for i in tqdm(range(100)):
    for d in [10,25,50,100,250]:
        selected_indices = np.random.choice(range(len(first_level_index)), d, replace = False)
        assets = first_level_index[selected_indices]
        nb_asset = len(assets)
        df = df_all[assets]
        X = df.values

        ### Fit Student-t mixture (2 components) to chosen stock returns
        n_sm = 2
        SM_model = sm.StudentMixture(n_components=n_sm, fixed_dofs = True, dofs_init = [2.5,4]).fit(X)

        # ERC
        budgets = np.ones(nb_asset)/nb_asset 
        # Expected Shortfall alpha
        alpha = .95 

        SM_theta, optim_res = StudentMixtureExpectedShortfall(SM_model).solve_risk_budgeting(budgets, alpha, on_simplex=False, kappa=1, method=None, maxiter=15000)
        VaR_port = StudentMixtureExpectedShortfall(SM_model).value_at_risk(SM_theta, alpha)
        ES_port = StudentMixtureExpectedShortfall(SM_model).expected_shortfall(SM_theta, alpha)
        risk_contribs = SM_theta * StudentMixtureExpectedShortfall(SM_model).expected_shortfall_gradient(SM_theta, alpha)

        optimal_loss = StudentMixtureExpectedShortfall(SM_model).expected_shortfall(optim_res.x, alpha) - np.dot(budgets, np.log(optim_res.x))

        n_val=1000000
        X = SM_model.rvs(n_val)
        np.random.shuffle(X)

        gamma = gamma_tamed[d]
        gamma_sgd_ = gamma_sgd[d]

        y_sgd = budgets / np.std(X, axis=0)
        xi_sgd = 0
        c_sgd = .65

        y_sgd_tamed = budgets / np.std(X, axis=0)
        xi_sgd_tamed = 0
        c_sgd_tamed = .65

        y_smd = budgets / np.std(X, axis=0)
        xi_smd = 0
        c_smd = .65
        M = 100

        y_sgd_s = [y_sgd]
        xi_sgd_s = [xi_sgd]
        y_bar_sgd_s = [y_sgd]

        y_sgd_tamed_s = [y_sgd_tamed]
        xi_sgd_tamed_s = [xi_sgd_tamed]
        y_bar_sgd_tamed_s = [y_sgd_tamed]

        y_smd_s = [y_smd]
        xi_smd_s = [xi_smd]
        y_bar_smd_s = [y_smd]

        error_sgd = []
        error_sgd_tamed = []
        error_smd = []

        norm_sgd = []
        norm_sgd_tamed = []
        norm_smd = []

        freq_error = 100000

        for k in range(1, n_val):
            x = X[k]
            
            ### SGD
            # gradient
            step_size_sgd = gamma_sgd_/k**c_sgd
            indicator_sgd = -np.dot(y_sgd, x) - xi_sgd >= 0
            grad_y_sgd = -x/(1-alpha)*indicator_sgd - budgets/y_sgd
            grad_xi_sgd = 1 - (1 / (1 - alpha)) * indicator_sgd

            #descent
            y_sgd = y_sgd - step_size_sgd*grad_y_sgd
            y_sgd = np.where(y_sgd <= 0, 1e-04, y_sgd)
            xi_sgd = xi_sgd - step_size_sgd*grad_xi_sgd
            # y_bar_sgd_numerator += y_sgd*step_size_sgd
            # y_bar_sgd_denominator += step_size_sgd

            ### SGD tamed
            # gradient
            step_size_sgd_tamed = gamma/k**c_sgd_tamed
            indicator_sgd_tamed = -np.dot(y_sgd_tamed, x) - xi_sgd_tamed >= 0
            grad_y_sgd_tamed = -x/(1-alpha)*indicator_sgd_tamed - budgets/y_sgd_tamed
            grad_xi_sgd_tamed = 1 - (1 / (1 - alpha)) * indicator_sgd_tamed

            #descent
            y_sgd_tamed_min = min(min(y_sgd_tamed),1)
            y_sgd_tamed = y_sgd_tamed - step_size_sgd*grad_y_sgd_tamed*y_sgd_tamed_min
            y_sgd_tamed = np.where(y_sgd_tamed <= 0, 1e-04, y_sgd_tamed)
            xi_sgd_tamed = xi_sgd_tamed - step_size_sgd_tamed*grad_xi_sgd_tamed
            # y_bar_sgd_numerator += y_sgd*step_size_sgd
            # y_bar_sgd_denominator += step_size_sgd


            ### SMD
            # gradient
            step_size_smd = gamma/k**c_smd
            indicator_smd = -np.dot(y_smd, x) - xi_smd >= 0
            grad_y_smd = -x/(1-alpha)*indicator_smd - budgets/y_smd
            grad_xi_smd = 1 - (1 / (1 - alpha)) * indicator_smd

            y_smd_min = min(min(y_smd),1)
            y_smd = y_smd*np.exp(-step_size_smd*y_smd_min*grad_y_smd)
            xi_smd = xi_smd - step_size_smd*grad_xi_smd
            sum_y_smd = np.sum(y_smd)
            if sum_y_smd>M:
                y_smd = M/sum_y_smd*y_smd

            if k%freq_error==0:
                error_sgd.append(StudentMixtureExpectedShortfall(SM_model).expected_shortfall(y_sgd, alpha) - np.dot(budgets, np.log(y_sgd)) - optimal_loss)
                error_sgd_tamed.append(StudentMixtureExpectedShortfall(SM_model).expected_shortfall(y_sgd_tamed, alpha) - np.dot(budgets, np.log(y_sgd_tamed)) - optimal_loss)
                error_smd.append(StudentMixtureExpectedShortfall(SM_model).expected_shortfall(y_smd, alpha) - np.dot(budgets, np.log(y_smd)) - optimal_loss)

                norm_sgd.append(np.mean(abs(SM_theta-y_sgd/y_sgd.sum())))
                norm_sgd_tamed.append(np.mean(abs(SM_theta-y_sgd_tamed/y_sgd_tamed.sum())))
                norm_smd.append(np.mean(abs(SM_theta-y_smd/y_smd.sum())))

        errors_sgd[d].append(error_sgd)
        errors_sgd_tamed[d].append(error_sgd_tamed)
        errors_smd[d].append(error_smd)

        norms_sgd[d].append(norm_sgd)
        norms_sgd_tamed[d].append(norm_sgd_tamed)
        norms_smd[d].append(norm_smd)

        np.save(str(d)+'_sgd_errors.npy', np.array(errors_sgd[d]))
        np.save(str(d)+'_smd_errors.npy', np.array(errors_smd[d]))     
        np.save(str(d)+'_sgd_tamed_errors.npy', np.array(errors_sgd_tamed[d]))

        np.save(str(d)+'_sgd_norms.npy', np.array(norms_sgd[d]))
        np.save(str(d)+'_smd_norms.npy', np.array(norms_smd[d]))
        np.save(str(d)+'_sgd_tamed_norms.npy', np.array(norms_sgd_tamed[d]))


In [ ]:
gamma_sgd = {10:5, 25:1, 50:.5, 100:.25, 250:.1}
gamma_tamed = {10:1, 25:2.5, 50:5, 100:10, 250:25}

errors_sgd = {10: [], 25: [], 50: [], 100: [], 250: []}
errors_sgd_tamed = {10: [], 25: [], 50: [], 100: [], 250: []}
errors_smd = {10: [], 25: [], 50: [], 100: [], 250: []}

norms_sgd = {10: [], 25: [], 50: [], 100: [], 250: []}
norms_sgd_tamed = {10: [], 25: [], 50: [], 100: [], 250: []}
norms_smd = {10: [], 25: [], 50: [], 100: [], 250: []}

for i in tqdm(range(100)):
    for d in [10,25,50,100,250]:
        selected_indices = np.random.choice(range(len(first_level_index)), d, replace = False)
        assets = first_level_index[selected_indices]
        nb_asset = len(assets)
        df = df_all[assets]
        X = df.values

        ### Fit Student-t mixture (2 components) to chosen stock returns
        n_sm = 2
        SM_model = sm.StudentMixture(n_components=n_sm, fixed_dofs = True, dofs_init = [2.5,4]).fit(X)

        # ERC
        budgets = np.ones(nb_asset)/nb_asset 
        # Expected Shortfall alpha
        alpha = .95 

        SM_theta, optim_res = StudentMixtureExpectedShortfall(SM_model).solve_risk_budgeting(budgets, alpha, on_simplex=False, kappa=1, method=None, maxiter=15000)
        VaR_port = StudentMixtureExpectedShortfall(SM_model).value_at_risk(SM_theta, alpha)
        ES_port = StudentMixtureExpectedShortfall(SM_model).expected_shortfall(SM_theta, alpha)
        risk_contribs = SM_theta * StudentMixtureExpectedShortfall(SM_model).expected_shortfall_gradient(SM_theta, alpha)

        optimal_loss = StudentMixtureExpectedShortfall(SM_model).expected_shortfall(optim_res.x, alpha) - np.dot(budgets, np.log(optim_res.x))

        n_val=1000000
        X = SM_model.rvs(n_val)
        np.random.shuffle(X)

        gamma = gamma_tamed[d]
        gamma_sgd_ = gamma_sgd[d]

        y_sgd = budgets / np.std(X, axis=0)
        xi_sgd = 0
        c_sgd = .65

        y_sgd_tamed = budgets / np.std(X, axis=0)
        xi_sgd_tamed = 0
        c_sgd_tamed = .65

        y_smd = budgets / np.std(X, axis=0)
        xi_smd = 0
        c_smd = .65
        M = 100

        y_sgd_s = [y_sgd]
        xi_sgd_s = [xi_sgd]
        y_bar_sgd_s = [y_sgd]

        y_sgd_tamed_s = [y_sgd_tamed]
        xi_sgd_tamed_s = [xi_sgd_tamed]
        y_bar_sgd_tamed_s = [y_sgd_tamed]

        y_smd_s = [y_smd]
        xi_smd_s = [xi_smd]
        y_bar_smd_s = [y_smd]

        error_sgd = []
        error_sgd_tamed = []
        error_smd = []

        norm_sgd = []
        norm_sgd_tamed = []
        norm_smd = []

        freq_error = 100000

        for k in range(1, n_val):
            x = X[k]
            
            ### SGD
            # gradient
            step_size_sgd = gamma_sgd_/k**c_sgd
            indicator_sgd = -np.dot(y_sgd, x) - xi_sgd >= 0
            grad_y_sgd = -x/(1-alpha)*indicator_sgd - budgets/y_sgd
            grad_xi_sgd = 1 - (1 / (1 - alpha)) * indicator_sgd

            #descent
            y_sgd = y_sgd - step_size_sgd*grad_y_sgd
            y_sgd = np.where(y_sgd <= 0, 1e-04, y_sgd)
            xi_sgd = xi_sgd - step_size_sgd*grad_xi_sgd
            # y_bar_sgd_numerator += y_sgd*step_size_sgd
            # y_bar_sgd_denominator += step_size_sgd

            ### SGD tamed
            # gradient
            step_size_sgd_tamed = gamma/k**c_sgd_tamed
            indicator_sgd_tamed = -np.dot(y_sgd_tamed, x) - xi_sgd_tamed >= 0
            grad_y_sgd_tamed = -x/(1-alpha)*indicator_sgd_tamed - budgets/y_sgd_tamed
            grad_xi_sgd_tamed = 1 - (1 / (1 - alpha)) * indicator_sgd_tamed

            #descent
            y_sgd_tamed_min = min(min(y_sgd_tamed),1)
            y_sgd_tamed = y_sgd_tamed - step_size_sgd*grad_y_sgd_tamed*y_sgd_tamed_min
            y_sgd_tamed = np.where(y_sgd_tamed <= 0, 1e-04, y_sgd_tamed)
            xi_sgd_tamed = xi_sgd_tamed - step_size_sgd_tamed*grad_xi_sgd_tamed
            # y_bar_sgd_numerator += y_sgd*step_size_sgd
            # y_bar_sgd_denominator += step_size_sgd


            ### SMD
            # gradient
            step_size_smd = gamma/k**c_smd
            indicator_smd = -np.dot(y_smd, x) - xi_smd >= 0
            grad_y_smd = -x/(1-alpha)*indicator_smd - budgets/y_smd
            grad_xi_smd = 1 - (1 / (1 - alpha)) * indicator_smd

            y_smd_min = min(min(y_smd),1)
            y_smd = y_smd*np.exp(-step_size_smd*y_smd_min*grad_y_smd)
            xi_smd = xi_smd - step_size_smd*grad_xi_smd
            sum_y_smd = np.sum(y_smd)
            if sum_y_smd>M:
                y_smd = M/sum_y_smd*y_smd

            if k%freq_error==0:
                error_sgd.append(StudentMixtureExpectedShortfall(SM_model).expected_shortfall(y_sgd, alpha) - np.dot(budgets, np.log(y_sgd)) - optimal_loss)
                error_sgd_tamed.append(StudentMixtureExpectedShortfall(SM_model).expected_shortfall(y_sgd_tamed, alpha) - np.dot(budgets, np.log(y_sgd_tamed)) - optimal_loss)
                error_smd.append(StudentMixtureExpectedShortfall(SM_model).expected_shortfall(y_smd, alpha) - np.dot(budgets, np.log(y_smd)) - optimal_loss)

                norm_sgd.append(np.mean(abs(SM_theta-y_sgd/y_sgd.sum())))
                norm_sgd_tamed.append(np.mean(abs(SM_theta-y_sgd_tamed/y_sgd_tamed.sum())))
                norm_smd.append(np.mean(abs(SM_theta-y_smd/y_smd.sum())))

        errors_sgd[d].append(error_sgd)
        errors_sgd_tamed[d].append(error_sgd_tamed)
        errors_smd[d].append(error_smd)

        norms_sgd[d].append(norm_sgd)
        norms_sgd_tamed[d].append(norm_sgd_tamed)
        norms_smd[d].append(norm_smd)

        np.save(str(d)+'_sgd_errors_convergence.npy', np.array(errors_sgd[d]))
        np.save(str(d)+'_smd_errors_convergence.npy', np.array(errors_smd[d]))     
        np.save(str(d)+'_sgd_tamed_errors_convergence.npy', np.array(errors_sgd_tamed[d]))

        np.save(str(d)+'_sgd_norms_convergence.npy', np.array(norms_sgd[d]))
        np.save(str(d)+'_smd_norms_convergence.npy', np.array(norms_smd[d]))
        np.save(str(d)+'_sgd_tamed_norms_convergence.npy', np.array(norms_sgd_tamed[d]))

In [5]:
## MAD, Volatility, Variantile
from scipy.optimize import minimize

def rb_vol(covariance_matrix, budgets):
    def objective(x):
        return (np.dot(x,np.dot(covariance_matrix, x)) - np.dot(budgets, np.log(x)))
    x0 = np.ones(len(budgets))
    bnds = [(1e-8,None) for _ in range(len(budgets))]
    res = minimize(objective, x0, bounds=bnds)
    return res.x, res.fun

In [ ]:
## MAD

norms_sgd = {10: [], 25: [], 50: [], 100: [], 250: []}
norms_smd = {10: [], 25: [], 50: [], 100: [], 250: []}


for i in tqdm(range(100)):
    for d in [10,25,50,100,250]:

        selected_indices = np.random.choice(range(len(first_level_index)), d, replace = False)
        assets = first_level_index[selected_indices]
        nb_asset = len(assets)
        df = df_all[assets]
        X = df.values

        ### Fit Student-t mixture (1 components) to chosen stock returns
        n_sm = 1
        SM_model = sm.StudentMixture(n_components=n_sm, fixed_dofs = True, dofs_init = [4]).fit(X)
        SM_model.locations_ = np.array([np.zeros(nb_asset)])

        # ERC
        budgets = np.ones(nb_asset)/nb_asset 

        y_ref, _ = rb_vol(SM_model.scales_[0], budgets)
        SM_theta = y_ref/y_ref.sum()

        n_val=500000
        X = SM_model.rvs(n_val)
        np.random.shuffle(X)

        gamma_sgd = 750
        gamma_smd = 5

        # y_sgd = d * budgets / np.std(X, axis=0)
        y_sgd = np.ones(d)*100
        xi_sgd = 0
        c_sgd = .65

        # y_smd = d * budgets / np.std(X, axis=0)
        y_smd = np.ones(d)*100
        xi_smd = 0
        c_smd = .65
        M = 100000

        y_sgd_s = [y_sgd]
        xi_sgd_s = [xi_sgd]
        y_bar_sgd_s = [y_sgd]

        y_smd_s = [y_smd]
        xi_smd_s = [xi_smd]
        y_bar_smd_s = [y_smd]

        norm_sgd = []
        norm_smd = []

        polyak = .2
                
        y_smd_final = 0
        y_sgd_final = 0

        for k in range(1, n_val):
            x = X[k]

            ### SGD
            # gradient
            step_size_sgd = gamma_sgd/k**c_sgd
            loss_sgd = -np.dot(y_sgd, x)
            indicator_pos_sgd = (loss_sgd - xi_sgd >= 0)*1
            indicator_neg_sgd = 1 - indicator_pos_sgd
            grad_y_sgd = (indicator_neg_sgd - indicator_pos_sgd)*x - d*budgets/y_sgd
            grad_xi_sgd = indicator_neg_sgd - indicator_pos_sgd

            #descent
            y_sgd = y_sgd - step_size_sgd*grad_y_sgd*min(min(y_sgd),1)
            y_sgd = np.where(y_sgd <= 0, 1, y_sgd)
            xi_sgd = xi_sgd - step_size_sgd*grad_xi_sgd
            # y_bar_sgd_numerator += y_sgd*step_size_sgd
            # y_bar_sgd_denominator += step_size_sgd

            ### SMD
            # gradient
            step_size_smd = gamma_smd/k**c_smd
            loss_smd = -np.dot(y_smd, x)
            indicator_pos_smd = (loss_smd - xi_smd >= 0)*1
            indicator_neg_smd = 1-indicator_pos_smd
            grad_y_smd = (indicator_neg_smd - indicator_pos_smd)*x - d*budgets/y_smd 
            grad_xi_smd = indicator_neg_smd - indicator_pos_smd

            y_smd_min = min(min(y_smd),1)
            y_smd = y_smd*np.exp(-step_size_smd*y_smd_min*grad_y_smd)
            xi_smd = xi_smd - step_size_smd*grad_xi_smd
            sum_y_smd = np.sum(y_smd)
            if sum_y_smd>M:
                y_smd = M/sum_y_smd*y_smd

            j=0
            if k>int(n_val - n_val*polyak):
                y_smd_final+=y_smd
                y_sgd_final+=y_sgd
                j+=1
            
        theta_sgd = y_sgd_final/j
        theta_sgd = theta_sgd/theta_sgd.sum()
        theta_smd = y_smd_final/j
        theta_smd = theta_smd/theta_smd.sum()

        norms_sgd[d].append(np.mean(abs(SM_theta-theta_sgd)))
        norms_smd[d].append(np.mean(abs(SM_theta-theta_smd)))

        np.save(str(d)+'_sgd_norms_mad.npy', np.array(norms_sgd[d]))
        np.save(str(d)+'_smd_norms_mad.npy', np.array(norms_smd[d]))
        # print(norms_sgd)
        # print(norms_smd)

In [6]:
## Volatility

norms_sgd = {10: [], 25: [], 50: [], 100: [], 250: []}
norms_smd = {10: [], 25: [], 50: [], 100: [], 250: []}


for i in tqdm(range(100)):
    for d in [10,25,50,100,250]:

        selected_indices = np.random.choice(range(len(first_level_index)), d, replace = False)
        assets = first_level_index[selected_indices]
        nb_asset = len(assets)
        df = df_all[assets]
        X = df.values

        ### Fit Student-t mixture (1 components) to chosen stock returns
        n_sm = 1
        SM_model = sm.StudentMixture(n_components=n_sm, fixed_dofs = True, dofs_init = [4]).fit(X)
        SM_model.locations_ = np.array([np.zeros(nb_asset)])

        # ERC
        budgets = np.ones(nb_asset)/nb_asset 

        y_ref, _ = rb_vol(SM_model.scales_[0], budgets)
        SM_theta = y_ref/y_ref.sum()

        n_val=500000
        X = SM_model.rvs(n_val)
        np.random.shuffle(X)

        gamma_sgd = 3.2
        gamma_smd = .8

        y_sgd = d**.5 * budgets / np.std(X, axis=0)
        xi_sgd = 0
        c_sgd = .65

        y_smd = d**.5 * budgets / np.std(X, axis=0)
        xi_smd = 0
        c_smd = .65
        M = 10000

        y_sgd_s = [y_sgd]
        xi_sgd_s = [xi_sgd]
        y_bar_sgd_s = [y_sgd]

        y_smd_s = [y_smd]
        xi_smd_s = [xi_smd]
        y_bar_smd_s = [y_smd]

        norm_sgd = []
        norm_smd = []

        polyak = .2
                
        y_smd_final = 0
        y_sgd_final = 0

        for k in range(1, n_val):
            x = X[k]

            ### SGD
            # gradient
            step_size_sgd = gamma_sgd/k**c_sgd
            grad_y_sgd = 2 * (np.dot(y_sgd,x) - xi_sgd)*x - d*budgets/y_sgd
            grad_xi_sgd = -2 * (np.dot(y_sgd,x) - xi_sgd)

            #descent
            y_sgd = y_sgd - step_size_sgd*grad_y_sgd*min(min(y_sgd),1)
            y_sgd = np.where(y_sgd <= 0, 1, y_sgd)
            xi_sgd = xi_sgd - step_size_sgd*grad_xi_sgd
            # y_bar_sgd_numerator += y_sgd*step_size_sgd
            # y_bar_sgd_denominator += step_size_sgd

            ### SMD
            # gradient
            step_size_smd = gamma_smd/k**c_smd
            grad_y_smd = 2 * (np.dot(y_smd,x) - xi_smd)*x - d*budgets/y_smd
            grad_xi_smd = -2 * (np.dot(y_smd,x) - xi_smd)

            y_smd_min = min(min(y_smd),1)
            y_smd = y_smd*np.exp(-step_size_smd*y_smd_min*grad_y_smd)
            xi_smd = xi_smd - step_size_smd*grad_xi_smd
            sum_y_smd = np.sum(y_smd)
            if sum_y_smd>M:
                y_smd = M/sum_y_smd*y_smd


            j=0
            if k>int(n_val - n_val*polyak):
                y_smd_final+=y_smd
                y_sgd_final+=y_sgd
                j+=1
            
        theta_sgd = y_sgd_final/j
        theta_sgd = theta_sgd/theta_sgd.sum()
        theta_smd = y_smd_final/j
        theta_smd = theta_smd/theta_smd.sum()

        norms_sgd[d].append(np.mean(abs(SM_theta-theta_sgd)))
        norms_smd[d].append(np.mean(abs(SM_theta-theta_smd)))

        np.save(str(d)+'_sgd_norms_volatility.npy', np.array(norms_sgd[d]))
        np.save(str(d)+'_smd_norms_volatility.npy', np.array(norms_smd[d]))
        
        # print(norms_sgd)
        # print(norms_smd)

100%|██████████| 100/100 [4:20:53<00:00, 156.53s/it] 


In [ ]:
## Variantile

norms_sgd = {10: [], 25: [], 50: [], 100: [], 250: []}
norms_smd = {10: [], 25: [], 50: [], 100: [], 250: []}


for i in tqdm(range(100)):
    for d in [10,25,50,100,250]:

        selected_indices = np.random.choice(range(len(first_level_index)), d, replace = False)
        assets = first_level_index[selected_indices]
        nb_asset = len(assets)
        df = df_all[assets]
        X = df.values

        ### Fit Student-t mixture (1 components) to chosen stock returns
        n_sm = 1
        SM_model = sm.StudentMixture(n_components=n_sm, fixed_dofs = True, dofs_init = [4]).fit(X)
        SM_model.locations_ = np.array([np.zeros(nb_asset)])

        # ERC
        budgets = np.ones(nb_asset)/nb_asset 

        y_ref, _ = rb_vol(SM_model.scales_[0], budgets)
        SM_theta = y_ref/y_ref.sum()

        n_val=500000
        X = SM_model.rvs(n_val)
        np.random.shuffle(X)

        alpha = .75

        gamma_sgd = 8
        gamma_smd = 1

        y_sgd = np.ones(d)*100/d**.5
        xi_sgd = 0
        c_sgd = .65

        y_smd = np.ones(d)*100/d**.5
        xi_smd = 0
        c_smd = .65
        M = 10000
        
        y_sgd_s = [y_sgd]
        xi_sgd_s = [xi_sgd]
        y_bar_sgd_s = [y_sgd]

        y_smd_s = [y_smd]
        xi_smd_s = [xi_smd]
        y_bar_smd_s = [y_smd]

        norm_sgd = []
        norm_smd = []

        polyak = .2
                
        y_smd_final = 0
        y_sgd_final = 0

        for k in range(1, n_val):
            x = X[k]

            ### SGD
            # gradient
            step_size_sgd = gamma_sgd/k**c_sgd
            loss_sgd = -np.dot(y_sgd, x)
            indicator_pos_sgd = loss_sgd - xi_sgd >= 0
            indicator_neg_sgd = loss_sgd - xi_sgd < 0
            grad_y_sgd = -2 * alpha * (loss_sgd - xi_sgd) * x * indicator_pos_sgd + -2 * (1 - alpha) * (loss_sgd - xi_sgd) * x * indicator_neg_sgd - d*budgets/y_sgd
            grad_xi_sgd = -2 * alpha * (loss_sgd - xi_sgd) * indicator_pos_sgd + -2 * (1 - alpha) * (loss_sgd - xi_sgd) * indicator_neg_sgd

            #descent
            y_sgd = y_sgd - step_size_sgd*grad_y_sgd*min(min(y_sgd),1)
            y_sgd = np.where(y_sgd <= 0, 1, y_sgd)
            xi_sgd = xi_sgd - step_size_sgd*grad_xi_sgd
            # y_bar_sgd_numerator += y_sgd*step_size_sgd
            # y_bar_sgd_denominator += step_size_sgd

            ### SMD
            # gradient
            step_size_smd = gamma_smd/k**c_smd
            loss_smd = -np.dot(y_smd, x)
            indicator_pos_smd = loss_smd - xi_smd >= 0
            indicator_neg_smd = loss_smd - xi_smd < 0
            grad_y_smd = -2 * alpha * (loss_smd - xi_smd) * x * indicator_pos_smd + -2 * (1 - alpha) * (loss_smd - xi_smd) * x * indicator_neg_smd - d*budgets/y_smd
            grad_xi_smd = -2 * alpha * (loss_smd - xi_smd) * indicator_pos_smd + -2 * (1 - alpha) * (loss_smd - xi_smd) * indicator_neg_smd

            y_smd_min = min(min(y_smd),1)
            y_smd = y_smd*np.exp(-step_size_smd*y_smd_min*grad_y_smd)
            xi_smd = xi_smd - step_size_smd*grad_xi_smd
            sum_y_smd = np.sum(y_smd)
            if sum_y_smd>M:
                y_smd = M/sum_y_smd*y_smd


            j=0
            if k>int(n_val - n_val*polyak):
                y_smd_final+=y_smd
                y_sgd_final+=y_sgd
                j+=1
            
        theta_sgd = y_sgd_final/j
        theta_sgd = theta_sgd/theta_sgd.sum()
        theta_smd = y_smd_final/j
        theta_smd = theta_smd/theta_smd.sum()

        norms_sgd[d].append(np.mean(abs(SM_theta-theta_sgd)))
        norms_smd[d].append(np.mean(abs(SM_theta-theta_smd)))

        np.save(str(d)+'_sgd_norms_variantile.npy', np.array(norms_sgd[d]))
        np.save(str(d)+'_smd_norms_variantile.npy', np.array(norms_smd[d]))
        # print(norms_sgd)
        # print(norms_smd)

In [ ]:
## MAD
from scipy.optimize import minimize

def rb_vol(covariance_matrix, budgets):
    def objective(x):
        return (np.dot(x,np.dot(covariance_matrix, x)) - np.dot(budgets, np.log(x)))
    x0 = np.ones(len(budgets))
    bnds = [(1e-8,None) for _ in range(len(budgets))]
    res = minimize(objective, x0, bounds=bnds)
    return res.x, res.fun

gamma_ = {10:20, 25:20, 50:20, 100:20, 250:20}

norms_sgd = {10: [], 25: [], 50: [], 100: [], 250: []}
norms_smd = {10: [], 25: [], 50: [], 100: [], 250: []}


for i in tqdm(range(100)):
    for d in [10,25,50,100,250]:

        selected_indices = np.random.choice(range(len(first_level_index)), d, replace = False)
        assets = first_level_index[selected_indices]
        nb_asset = len(assets)
        df = df_all[assets]
        X = df.values

        ### Fit Student-t mixture (1 components) to chosen stock returns
        n_sm = 1
        SM_model = sm.StudentMixture(n_components=n_sm, fixed_dofs = True, dofs_init = [4]).fit(X)
        SM_model.locations_ = np.array([np.zeros(nb_asset)])

        # ERC
        budgets = np.ones(nb_asset)/nb_asset 
         
        y_ref, _ = rb_vol(SM_model.scales_[0], budgets)
        SM_theta = y_ref/y_ref.sum()

        n_val=1000000
        X = SM_model.rvs(n_val)
        np.random.shuffle(X)

        gamma = gamma_[d]

        y_sgd = budgets / np.std(X, axis=0)
        xi_sgd = 0
        c_sgd = .65

        y_smd = budgets / np.std(X, axis=0)
        xi_smd = 0
        c_smd = .65
        M = 200

        y_sgd_s = [y_sgd]
        xi_sgd_s = [xi_sgd]
        y_bar_sgd_s = [y_sgd]

        y_smd_s = [y_smd]
        xi_smd_s = [xi_smd]
        y_bar_smd_s = [y_smd]

        norm_sgd = []
        norm_smd = []

        polyak = .2
                
        y_smd_final = 0
        y_sgd_final = 0

        for k in range(1, n_val):
            x = X[k]

            ### SGD
            # gradient
            step_size_sgd = gamma/k**c_sgd
            loss_sgd = -np.dot(y_sgd, x)
            indicator_pos_sgd = (loss_sgd - xi_sgd >= 0)*1
            indicator_neg_sgd = 1 - indicator_pos_sgd
            grad_y_sgd = (indicator_neg_sgd - indicator_pos_sgd)*x - budgets/y_sgd
            grad_xi_sgd = indicator_neg_sgd - indicator_pos_sgd

            #descent
            y_sgd = y_sgd - step_size_sgd*grad_y_sgd*min(min(y_sgd),1)
            y_sgd = np.where(y_sgd <= 0, 1e-04, y_sgd)
            xi_sgd = xi_sgd - step_size_sgd*grad_xi_sgd
            # y_bar_sgd_numerator += y_sgd*step_size_sgd
            # y_bar_sgd_denominator += step_size_sgd

            ### SMD
            # gradient
            step_size_smd = gamma/k**c_smd
            loss_smd = -np.dot(y_smd, x)
            indicator_pos_smd = (loss_smd - xi_smd >= 0)*1
            indicator_neg_smd = 1-indicator_pos_smd
            grad_y_smd = (indicator_neg_smd - indicator_pos_smd)*x - budgets/y_smd 
            grad_xi_smd = indicator_neg_smd - indicator_pos_smd

            y_smd_min = min(min(y_smd),1)
            y_smd = y_smd*np.exp(-step_size_smd*y_smd_min*grad_y_smd)
            xi_smd = xi_smd - step_size_smd*grad_xi_smd
            sum_y_smd = np.sum(y_smd)
            if sum_y_smd>M:
                y_smd = M/sum_y_smd*y_smd
            
            j=0
            if k>int(n_val - n_val*polyak):
                y_smd_final+=y_smd
                y_sgd_final+=y_sgd
                j+=1
            
        theta_sgd = y_sgd_final/j
        theta_sgd = theta_sgd/theta_sgd.sum()
        theta_smd = y_smd_final/j
        theta_smd = theta_smd/theta_smd.sum()

        norms_sgd[d].append(np.mean(abs(SM_theta-theta_sgd)))
        norms_smd[d].append(np.mean(abs(SM_theta-theta_smd)))

        np.save(str(d)+'_sgd_norms_mad.npy', np.array(norms_sgd[d]))
        np.save(str(d)+'_smd_norms_mad.npy', np.array(norms_smd[d]))

In [ ]:
## MAD
from scipy.optimize import minimize

def rb_vol(covariance_matrix, budgets):
    def objective(x):
        return (np.dot(x,np.dot(covariance_matrix, x)) - np.dot(budgets, np.log(x)))
    x0 = np.ones(len(budgets))
    bnds = [(1e-8,None) for _ in range(len(budgets))]
    res = minimize(objective, x0, bounds=bnds)
    return res.x, res.fun

gamma_ = {10:20, 25:20, 50:20, 100:20, 250:20}

norms_sgd = {10: [], 25: [], 50: [], 100: [], 250: []}
norms_smd = {10: [], 25: [], 50: [], 100: [], 250: []}


for i in tqdm(range(100)):
    for d in [10,25,50,100,250]:

        selected_indices = np.random.choice(range(len(first_level_index)), d, replace = False)
        assets = first_level_index[selected_indices]
        nb_asset = len(assets)
        df = df_all[assets]
        X = df.values

        ### Fit Student-t mixture (1 components) to chosen stock returns
        n_sm = 1
        SM_model = sm.StudentMixture(n_components=n_sm, fixed_dofs = True, dofs_init = [4]).fit(X)
        SM_model.locations_ = np.array([np.zeros(nb_asset)])

        # ERC
        budgets = np.ones(nb_asset)/nb_asset 
         
        y_ref, _ = rb_vol(SM_model.scales_[0], budgets)
        SM_theta = y_ref/y_ref.sum()

        n_val=1000000
        X = SM_model.rvs(n_val)
        np.random.shuffle(X)

        gamma = gamma_[d]

        y_sgd = budgets / np.std(X, axis=0)
        xi_sgd = 0
        c_sgd = .65

        y_smd = budgets / np.std(X, axis=0)
        xi_smd = 0
        c_smd = .65
        M = 200

        y_sgd_s = [y_sgd]
        xi_sgd_s = [xi_sgd]
        y_bar_sgd_s = [y_sgd]

        y_smd_s = [y_smd]
        xi_smd_s = [xi_smd]
        y_bar_smd_s = [y_smd]

        norm_sgd = []
        norm_smd = []

        polyak = .2
                
        y_smd_final = 0
        y_sgd_final = 0

        for k in range(1, n_val):
            x = X[k]

            ### SGD
            # gradient
            step_size_sgd = gamma/k**c_sgd
            loss_sgd = -np.dot(y_sgd, x)
            indicator_pos_sgd = (loss_sgd - xi_sgd >= 0)*1
            indicator_neg_sgd = 1 - indicator_pos_sgd
            grad_y_sgd = (indicator_neg_sgd - indicator_pos_sgd)*x - budgets/y_sgd
            grad_xi_sgd = indicator_neg_sgd - indicator_pos_sgd

            #descent
            y_sgd = y_sgd - step_size_sgd*grad_y_sgd*min(min(y_sgd),1)
            y_sgd = np.where(y_sgd <= 0, 1e-04, y_sgd)
            xi_sgd = xi_sgd - step_size_sgd*grad_xi_sgd
            # y_bar_sgd_numerator += y_sgd*step_size_sgd
            # y_bar_sgd_denominator += step_size_sgd

            ### SMD
            # gradient
            step_size_smd = gamma/k**c_smd
            loss_smd = -np.dot(y_smd, x)
            indicator_pos_smd = (loss_smd - xi_smd >= 0)*1
            indicator_neg_smd = 1-indicator_pos_smd
            grad_y_smd = (indicator_neg_smd - indicator_pos_smd)*x - budgets/y_smd 
            grad_xi_smd = indicator_neg_smd - indicator_pos_smd

            y_smd_min = min(min(y_smd),1)
            y_smd = y_smd*np.exp(-step_size_smd*y_smd_min*grad_y_smd)
            xi_smd = xi_smd - step_size_smd*grad_xi_smd
            sum_y_smd = np.sum(y_smd)
            if sum_y_smd>M:
                y_smd = M/sum_y_smd*y_smd
            
            j=0
            if k>int(n_val - n_val*polyak):
                y_smd_final+=y_smd
                y_sgd_final+=y_sgd
                j+=1
            
        theta_sgd = y_sgd_final/j
        theta_sgd = theta_sgd/theta_sgd.sum()
        theta_smd = y_smd_final/j
        theta_smd = theta_smd/theta_smd.sum()

        norms_sgd[d].append(np.mean(abs(SM_theta-theta_sgd)))
        norms_smd[d].append(np.mean(abs(SM_theta-theta_smd)))

        np.save(str(d)+'_sgd_norms_mad.npy', np.array(norms_sgd[d]))
        np.save(str(d)+'_smd_norms_mad.npy', np.array(norms_smd[d]))

In [ ]:
stop

In [ ]:
### Import stock price data for model calibration
freq = 'B'
assets = ['JPM UN Equity', 'PFE UN Equity', 'XOM UN Equity']
nb_asset = len(assets)

df_all = pd.concat([pd.read_excel('Data/SP_RC.xlsx',index_col=0, header=[0,1]), pd.read_excel('Data/SP_RC2.xlsx',index_col=0, header=[0,1])])
df_all = df_all.dropna(how='all').dropna(axis=1)
df_all = df_all.pct_change().dropna()
df_all = df_all.replace([np.inf, -np.inf], np.nan)
df_all = df_all[~df_all.index.duplicated(keep='first')]
df = df_all[assets]

X = df.values
print(X.shape)

In [ ]:
# selected_indices = np.random.choice(range(len(first_level_index)), 50, replace = False)
# assets = first_level_index[selected_indices]
# nb_asset = len(assets)
# df = df_all[assets]
# X = df.values


In [ ]:
### Fit Student-t mixture (2 components) to chosen stock returns
n_sm = 2
SM_model = sm.StudentMixture(n_components=n_sm, fixed_dofs = True, dofs_init = [2.5,4]).fit(X)

In [ ]:
### Print model parameters 
print("==== Number of assets ====")
print(nb_asset)
print("==== Number of mixture components ====")
print(n_sm)
print("==== Weights (probability) parameters ====")
print(SM_model.weights_)
print("==== Degree of freedom parameters ====")
print(SM_model.dofs_)
print("==== Location parameters ====")
print(SM_model.locations_)
print("==== Scale parameters ====")
print(SM_model.scales_)

In [ ]:
### Compute the Risk Budgeting portfolio under the above model

# ERC
budgets = np.ones(nb_asset)/nb_asset 
# Expected Shortfall alpha
alpha = .95 

SM_theta, optim_res = StudentMixtureExpectedShortfall(SM_model).solve_risk_budgeting(budgets, alpha, on_simplex=False, kappa=1, method=None, maxiter=15000)
VaR_port = StudentMixtureExpectedShortfall(SM_model).value_at_risk(SM_theta, alpha)
ES_port = StudentMixtureExpectedShortfall(SM_model).expected_shortfall(SM_theta, alpha)
risk_contribs = SM_theta * StudentMixtureExpectedShortfall(SM_model).expected_shortfall_gradient(SM_theta, alpha)

print('==== Target Risk Bugdets ====')
print(np.round(budgets, 4))
print('==== $y^*$ ====')
print(np.round(optim_res.x, 4))

print('==== Risk Budgeting portfolio ($u^*$) ====')
print(np.round(SM_theta, 4))
print('==== VaR of the portfolio ====')
print(np.round(VaR_port, 4))
print('==== Expected Shortfall of the portfolio ====')
print(np.round(ES_port, 4))
print('==== Risk Contributions ====')
print(np.round(risk_contribs, 5))
print('==== Risk Contributions (normalized) ====')
print(np.round(risk_contribs/sum(risk_contribs), 2))

In [ ]:
StudentMixtureExpectedShortfall(SM_model).solve_risk_budgeting(budgets, alpha, on_simplex=False, kappa=1, method=None, maxiter=15000)

In [ ]:
optimal_loss = StudentMixtureExpectedShortfall(SM_model).expected_shortfall(optim_res.x, alpha) - np.dot(budgets, np.log(optim_res.x))
optimal_loss

### SMD vs. SGD - Error analysis

In [ ]:
np.random.seed(0)

errors_sgd = []
errors_smd = []

for i in tqdm(range(100)):

    n_val=1000000
    X = SM_model.rvs(n_val)
    np.random.shuffle(X)

    alpha = .95
    gamma = 10

    y_sgd = budgets / np.std(X, axis=0)
    xi_sgd = 0
    c_sgd = .65

    y_smd = budgets / np.std(X, axis=0)
    xi_smd = 0
    c_smd = .8
    M = 500

    y_sgd_s = [y_sgd]
    xi_sgd_s = [xi_sgd]
    y_bar_sgd_s = [y_sgd]
    y_bar_sgd_numerator = 0
    y_bar_sgd_denominator = 0

    y_smd_s = [y_smd]
    xi_smd_s = [xi_smd]
    y_bar_smd_s = [y_smd]
    y_bar_smd_numerator = 0
    y_bar_smd_denominator = 0

    error_sgd = []
    error_smd = []

    freq_error = 50

    for k in range(1, n_val):
        x = X[k]
        
        ### SGD
        # gradient
        step_size_sgd = gamma/k**c_sgd
        indicator_sgd = -np.dot(y_sgd, x) - xi_sgd >= 0
        grad_y_sgd = -x/(1-alpha)*indicator_sgd - budgets/y_sgd
        grad_xi_sgd = 1 - (1 / (1 - alpha)) * indicator_sgd

        #descent
        y_sgd = y_sgd - step_size_sgd*grad_y_sgd
        y_sgd = np.where(y <= 0, 1e-04, y_sgd)
        xi_sgd = xi_sgd - step_size_sgd*grad_xi_sgd
        y_bar_sgd_numerator += y_sgd*step_size_sgd
        y_bar_sgd_denominator += step_size_sgd

        ### SMD
        # gradient
        step_size_smd = gamma/k**c_smd
        indicator_smd = -np.dot(y_smd, x) - xi_smd >= 0
        grad_y_smd = -x/(1-alpha)*indicator_smd - budgets/y_smd
        grad_xi_smd = 1 - (1 / (1 - alpha)) * indicator_smd

        y_smd_min = min(min(y_smd),1)
        y_smd = y_smd*np.exp(-step_size_smd*y_smd_min*grad_y_smd)
        xi_smd = xi_smd - step_size_smd*grad_xi_smd
        sum_y_smd = np.sum(y_smd)
        if sum_y_smd>M:
            y_smd = M/sum_y_smd*y_smd
        y_bar_smd_numerator += y_smd*step_size_smd
        y_bar_smd_denominator += step_size_smd

        # y_sgd_s.append(y_sgd)
        # xi_sgd_s.append(xi_sgd)
        # y_bar_sgd_s.append(y_bar_sgd_numerator/y_bar_sgd_denominator)

        # y_smd_s.append(y_smd)
        # xi_smd_s.append(xi_smd)
        # y_bar_smd_s.append(y_bar_smd_numerator/y_bar_smd_denominator)

        if k%freq_error==0:
            error_sgd.append(StudentMixtureExpectedShortfall(SM_model).expected_shortfall(y_sgd, alpha) - np.dot(budgets, np.log(y_sgd)) - optimal_loss)
            error_smd.append(StudentMixtureExpectedShortfall(SM_model).expected_shortfall(y_smd, alpha) - np.dot(budgets, np.log(y_smd)) - optimal_loss)
    
    errors_sgd.append(error_sgd)
    errors_smd.append(error_smd)

    np.save('sgd_errors.npy', np.array(errors_sgd))
    np.save('smd_errors.npy', np.array(errors_smd))

In [ ]:
errors_sgd = []
errors_smd = []
for i in tqdm(ns):
    errors_sgd.append(StudentMixtureExpectedShortfall(SM_model).expected_shortfall(y_sgd_s[i], alpha) - np.dot(budgets, np.log(y_sgd_s[i])) - optimal_loss)
    errors_smd.append(StudentMixtureExpectedShortfall(SM_model).expected_shortfall(y_smd_s[i], alpha) - np.dot(budgets, np.log(y_smd_s[i])) - optimal_loss)

In [ ]:
errors_sgd_bar = []
errors_smd_bar = []
for i in tqdm(ns):
    errors_sgd_bar.append(StudentMixtureExpectedShortfall(SM_model).expected_shortfall(y_bar_sgd_s[i], alpha) - np.dot(budgets, np.log(y_bar_sgd_s[i])) - optimal_loss)
    errors_smd_bar.append(StudentMixtureExpectedShortfall(SM_model).expected_shortfall(y_bar_smd_s[i], alpha) - np.dot(budgets, np.log(y_bar_smd_s[i])) - optimal_loss)

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(12,4))
ax[0].plot(ns, errors_sgd);
ax[0].plot(ns, errors_smd);
ax[0].set_yscale('log')

ax[1].plot(ns, errors_sgd_bar);
ax[1].plot(ns, errors_smd_bar);
ax[1].set_yscale('log')

In [ ]:
y_sgd_s[-1]/sum(y_sgd_s[-1])

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(15,4))
ax[0].plot(y_sgd_s);
ax[1].plot(y_smd_s);

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(15,4))
ax[0].plot(y_sgd_s);
ax[1].plot(y_smd_s);

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(15,4))
ax[0].plot(y_sgd_s);
ax[1].plot(y_smd_s);


In [ ]:
y_sgd_s[-1]

In [ ]:
y_smd_s[-1]

In [ ]:
%%time
# we define our risk budgeting problem for expected shortfall
rb = RiskBudgeting(risk_measure='expected_shortfall',
                   alpha=0.95,
                   budgets='ERC')
rb.solve(X, 
         epochs=1,
         minibatch_size=1,
         eta_0_y=10, 
         eta_0_t=10,
         c=0.65,
         polyak_ruppert=0.2,
         store=True)

print('==== True Risk Budgeting portfolio ====')
print(np.round(SM_theta, 4))
print('==== Risk Budgeting portfolio from SGD ====')
print(np.round(rb.x, 4))

plt.plot(rb.ys);

In [ ]:
%%time
M=30
rb_md = stoch_sol(X, M, n_val, nb_asset, budgets, n_epochs=1, gamma=10)

#### Error analysis

In [ ]:
y_s = np.array(rb.ys)
x,y,z = np.array(rb_md.y_bars).shape
y_s_md = np.array(rb_md.y_bars).reshape(x,z)
optimal_loss = StudentMixtureExpectedShortfall(SM_model).expected_shortfall(optim_res.x, alpha) - np.dot(budgets, np.log(optim_res.x))
optimal_loss

In [ ]:
errors = []
errors_md = []
for i in tqdm(range(0,len(y_s)-1,100)):
    errors.append(StudentMixtureExpectedShortfall(SM_model).expected_shortfall(y_s[i], alpha) - np.dot(budgets, np.log(y_s[i])) - optimal_loss)
    errors_md.append(StudentMixtureExpectedShortfall(SM_model).expected_shortfall(y_s_md[i], alpha) - np.dot(budgets, np.log(y_s_md[i])) - optimal_loss)

In [ ]:
len(y_s)

In [ ]:
len(y_s_md)

In [ ]:
plt.plot(range(0,len(y_s)-1,100), errors)
plt.plot(range(0,len(y_s)-1,100), errors_md)
plt.yscale('log')

In [ ]:
min(errors)

In [ ]:
y_s

In [ ]:
%%time
M=50
solution_md = stoch_sol(X, M, n_val, nb_asset, budgets, n_epochs=1)

In [ ]:
y_s_md = solution_md.ys.T

In [ ]:
errors_md = []
for i in tqdm(range(0,len(y_s_md),50)):
    errors_md.append(StudentMixtureExpectedShortfall(SM_model).expected_shortfall(y_s_md[i], alpha) - np.dot(budgets, np.log(y_s_md[i])) - optimal_loss)
    

In [ ]:
plt.plot(range(0,len(y_s_md),50), errors_md)
plt.yscale('log')